# Density ROM in Python

In [1]:
"""
Created on Sat Mar 28 20:03:33 2020
ROM in Python 
@author: maggiezhang
"""
# imports necessary for program 

import numpy as np
from scipy.io import loadmat 
import h5py
import pandas as pd 
import matplotlib.pyplot as plt 


In [2]:
#%% code begins
# Reduced dimension
r = 10

# Year of data
year = 2000

# Density data path
datapath = "/Users/maggiezhang/Desktop"

localSolarTimes = np.linspace(0,24,24)
latitudes = np.linspace(-87.5,87.5,20)
altitudes = np.linspace(100,800,36)
nofAlt = len(altitudes)
nofLst = len(localSolarTimes)
nofLat = len(latitudes)

In [9]:
# %% Processing Density Data 
densityDataFull = np.array([])
SWdataFull = np.array([])

datasource = 'JB2008'
if datasource == 'JB2008':
    denseData = np.load(datapath+'/JB_full.npy').T
    spacewData = h5py.File('/Users/maggiezhang/Dropbox/AtmosphereMachineLearning/DensityData/JB2008_1999_2010_ROM_r100.mat', 'r')
elif datasource == "MSISE":
    pass # finish later on

print(denseData.shape)
#print(denseData[:5,:5])
spacewData = spacewData['SWdataFull']
print(spacewData.shape)
#df = pd.DataFrame(np.transpose(spacewData), ['doy', 'UThrs', 'F10', 'F10B', 'S10', 'S10B', 'XM10', 'XM10B', 'Y10', 'Y10B', 'DSTDTC', 'GWRAS', 'SUN(1)', 'SUN(2)'])
#print(df.head())
#print(spacewData.shape)
#print(denseData.shape)

(17280, 105192)
(14, 103404)


In [ ]:
spacewData = h5py.File('/Users/maggiezhang/Dropbox/AtmosphereMachineLearning/DensityData/JB2008_1999_2010_ROM_r100.mat', 'r')
print(denseData.shape)
#print(denseData[:5,:5])
spacewData = spacewData['SWdataFull']
print(spacewData.shape)
df = pd.DataFrame(spacewData, ['doy', 'UThrs', 'F10', 'F10B', 'S10', 'S10B', 'XM10', 'XM10B', 'Y10', 'Y10B', 'DSTDTC', 'GWRAS', 'SUN(1)', 'SUN(2)'])
print(df.head())
print(spacewData.shape)
print(denseData.shape)

In [10]:
# %%Proper Orthogonal Decomp
#reduce the dimenion of density data 
densityDataMeanLog = np.log10(np.mean(denseData, axis = 1))
densityDataMeanLog = np.reshape(densityDataMeanLog, (17280, 1))
densityDataVarLog = np.log10(denseData) - densityDataMeanLog
print(densityDataMeanLog)
print(densityDataVarLog[0])

[[ -6.2589803]
 [ -6.2589774]
 [ -6.2589765]
 ...
 [-13.889839 ]
 [-13.892305 ]
 [-13.893965 ]]
[-0.12916136 -0.12919807 -0.12918425 ... -0.1437645  -0.14390707
 -0.14390755]


In [ ]:
# %%Singular Value Decomp 
u,d,v = np.linalg.svd(densityDataVarLog)
jb_svd = {"u":u, "d": d, "v": v}
with open('JB_SVD.txt', 'w') as outfile:
    json.dump(jb_svd, outfile)


In [9]:
# %%Mapping density onto principal components to obtain reduced order 
densityDataLogVarROM = np.matmul(np.transpose(u[:,0:r]), densityDataVarLog)
print(densityDataLogVarROM.shape)
print(densityDataLogVarROM[0])


(10, 8784)
[26.34354836 19.07466225 19.07950122 ... 11.47553681 17.21583568
 17.21861349]


In [14]:
#%% Dynamic Mode Decomp with control (DMD c)

startIndex = 0

X1 = densityDataLogVarROM[0:r, startIndex: -1]

minval = 0
maxval = 0
num = 0
for i in range(len(X1)):
    for j in range(len(X1[0])):
        if X1[i][j] < minval:
            minval = X1[i][j]
        if X1[i][j] > maxval:
            maxval = X1[i][j]
        if X1[i][j] <0:
            num+=1
print(minval)
print(maxval)
print(num)
print(X1.shape)
X2 = densityDataLogVarROM[0:r, startIndex+1:]
print(X2.shape)

if datasource == 'JB2008':
    # standard space weather inputs for the linear model
    U1 = np.transpose(spacewData[startIndex:-1,:])
    #print(U1.shape)
    # add next timesteps for better prediction
    U1 = np.append(U1,np.transpose(spacewData[startIndex+1:,10:11]), axis = 0)
    #print(U1[14:15, 0:5])
    U1 = np.append(U1,np.transpose(spacewData[startIndex+1:,2:3]), axis = 0)
    U1 = np.append(U1,np.transpose(spacewData[startIndex+1:,4:5]), axis = 0)
    U1 = np.append(U1,np.transpose(spacewData[startIndex+1:,6:7]), axis = 0)
    U1 = np.append(U1,np.transpose(spacewData[startIndex+1:,8:9]), axis = 0)
    #adding mixed terms
    #rand = np.matmul(np.transpose(spacewData[startIndex:-1,10:11]),np.transpose(spacewData[startIndex:-1,2:3]))
    #print(rand.shape)
    U1 = np.append(U1,(np.multiply(np.transpose(spacewData[startIndex:-1,10:11]),np.transpose(spacewData[startIndex:-1,2:3]))), axis = 0)
    U1 = np.append(U1,(np.multiply(np.transpose(spacewData[startIndex+1:,10:11]),np.transpose(spacewData[startIndex+1:,2:3]))), axis = 0)
    
    #adding quadratic DSTDTC 
    U1 = np.append(U1,np.transpose(np.square((spacewData[startIndex:-1,10:11]))), axis = 0)
    U1 = np.append(U1,np.transpose(np.square((spacewData[startIndex+1:,10:11]))), axis = 0)
    print(U1.shape)

elif datasource == "MSISE":
    pass # finish later on

-80.07955152266089
44.56890526056185
43837
(10, 8783)
(10, 8783)
(23, 8783)


In [1]:
#%% Linear Dynamical Model
#X2 = A*X1 + B*U1
#X2 = [A B]*[X1;U1]
#X2 = Phi*Om, where Phi=[A B] and Om=[X1;U1]
Om = np.append(X1, U1, axis = 0)
print(X1.shape)
print(U1.shape)
print(Om.shape)
print(X2.shape, "x2")
# Compute dynamic matrices:
# Phi = X2*pinv(Om)
Phi = np.matmul(X2,np.linalg.pinv(Om))
print(Phi.shape)
A = Phi[:r, :r]
print(A.shape)
B = Phi[:r, r:]
print(B.shape)

a,b = np.linalg.eig(A)
plt.figure()
plt.plot(a)

plt.figure()
plt.scatter(np.real(np.diagonal(b)),np.imag(np.diagonal(b)))


NameError: name 'np' is not defined

In [10]:
#%% Check avg accuracy of 1 prediction step
print(A.shape)
print(X1.shape)
print(B.shape)
print(U1.shape)
XfullPred = np.matmul(A,X1) + np.matmul(B,U1)
print(XfullPred.shape, "xfullpred")
errFullPred = abs(XfullPred-X2)

print(np.mean(np.transpose(errFullPred), axis = 0))

(10, 10)
(10, 8783)
(10, 23)
(23, 8783)
(10, 8783) xfullpred
[0.07860087 0.05975295 0.02202399 0.01114433 0.00914694 0.022638
 0.01945343 0.01248644 0.0051234  0.00637508]


In [12]:
#%% Predict the density of all timesteps 

startPred = 980
nofSteps = 10*24
Upred = U1[:,startPred - 1: startPred+nofSteps]
print(Upred.shape)
Xpred = X1[:, startPred-1:startPred]
print(Xpred.shape)
for i in range(nofSteps):
    Xpred = np.append(Xpred, np.matmul(A,Xpred[:,i:i+1]) + np.matmul(B, Upred[:,i:i+1]),axis = 1)
print(Xpred.shape, "xpred shape")
newdf = pd.DataFrame(Xpred)
print(newdf.head())
#print(Xpred.shape)
densityVarRomPred= np.matmul(u[:,:r], Xpred)
#print(densityVarRomPred[:5,:5])

matrixof10 = np.full((17280, 241), 10)
print((densityVarRomPred+densityDataMeanLog).shape)
densityRomPred = np.power(matrixof10,(densityVarRomPred+densityDataMeanLog))
print(densityRomPred[:5,:5])
print(denseData[:5,:5])

#densityRomPred
#print(densityRomPred[:5,:5])

densityRomCorrect = denseData[:, startPred-1: startPred+nofSteps]
diffDens = abs(densityRomCorrect-densityRomPred)/densityRomCorrect*100
print(diffDens.shape, "diff dense")
mse = 0
for i in range(len(diffDens)):
    for j in range(len(diffDens[0])):
        mse += diffDens[i][j]**2
mse = mse/(diffDens.shape[0]*diffDens.shape[1])
print(mse)
#print(diffDense[0])

(23, 241)
(10, 1)
(10, 241) xpred shape
        0         1         2         3         4         5         6    \
0  1.066298  1.044960  1.024180 -1.226791 -1.246778 -1.266273  0.950476   
1  6.067867  6.052582  6.038375  5.936549  5.923587  5.911587  5.989851   
2 -0.518561 -0.521581 -0.524306 -0.444767 -0.447005 -0.448980 -0.532206   
3  0.083819  0.083980  0.084066  0.227697  0.227731  0.227702  0.083241   
4  1.338552  1.339431  1.340196  1.367019  1.367623  1.368125  1.343167   

        7         8         9    ...        231        232        233  \
0  0.936926  0.923532 -8.694676  ...  27.627353  27.614588  29.411786   
1  5.980998  5.972839  5.587680  ...   5.737918   5.735353   5.789960   
2 -0.533219 -0.534078 -0.163679  ...  -0.614735  -0.616689  -0.639507   
3  0.082299  0.081349  0.682838  ...  -0.485411  -0.485902  -0.533280   
4  1.344253  1.345273  1.454300  ...   1.123871   1.123544   1.089716   

         234        235        236        237        238        239   